<a href="https://colab.research.google.com/github/redwankarimsony/hackerearth_employee_burnout/blob/main/HackerEarth_(Employee_Burnout_Challenge_2020).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import numpy as np, os
import pandas as pd
import matplotlib.pyplot as plt

import lightgbm as lgb
from sklearn.model_selection import train_test_split


HOME_DIR = '/content/'
DATA_DIR = os.path.join(HOME_DIR, 'data')

In [85]:
train_df = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
test_df = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))
sub = pd.read_csv(os.path.join(DATA_DIR, 'sample_submission.csv'))

## Utility Functions

In [86]:
def get_days(d0, d1):
    d0 = pd.to_datetime(d0)
    d1 = pd.to_datetime(d1)
    delta = d1 - d0
    return delta.days

In [87]:
train_df.head()

,Employee ID,Date of Joining,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate
0,fffe32003000360033003200,2008-09-30,Female,Service,No,2.0,3.0,3.8,0.16
1,fffe3700360033003500,2008-11-30,Male,Service,Yes,1.0,2.0,5.0,0.36
2,fffe31003300320037003900,2008-03-10,Female,Product,Yes,2.0,NaN,5.8,0.49
3,fffe32003400380032003900,2008-11-03,Male,Service,Yes,1.0,1.0,2.6,0.20
4,fffe31003900340031003600,2008-07-24,Female,Service,No,3.0,7.0,6.9,0.52


In [88]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22750 entries, 0 to 22749
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Employee ID           22750 non-null  object 
 1   Date of Joining       22750 non-null  object 
 2   Gender                22750 non-null  object 
 3   Company Type          22750 non-null  object 
 4   WFH Setup Available   22750 non-null  object 
 5   Designation           22750 non-null  float64
 6   Resource Allocation   21369 non-null  float64
 7   Mental Fatigue Score  20633 non-null  float64
 8   Burn Rate             21626 non-null  float64
dtypes: float64(4), object(5)
memory usage: 1.6+ MB


In [89]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12250 entries, 0 to 12249
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Employee ID           12250 non-null  object 
 1   Date of Joining       12250 non-null  object 
 2   Gender                12250 non-null  object 
 3   Company Type          12250 non-null  object 
 4   WFH Setup Available   12250 non-null  object 
 5   Designation           12250 non-null  float64
 6   Resource Allocation   12250 non-null  float64
 7   Mental Fatigue Score  12250 non-null  float64
dtypes: float64(3), object(5)
memory usage: 765.8+ KB


In [90]:
train_df.dropna(inplace=True)

In [91]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18590 entries, 0 to 22749
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Employee ID           18590 non-null  object 
 1   Date of Joining       18590 non-null  object 
 2   Gender                18590 non-null  object 
 3   Company Type          18590 non-null  object 
 4   WFH Setup Available   18590 non-null  object 
 5   Designation           18590 non-null  float64
 6   Resource Allocation   18590 non-null  float64
 7   Mental Fatigue Score  18590 non-null  float64
 8   Burn Rate             18590 non-null  float64
dtypes: float64(4), object(5)
memory usage: 1.4+ MB


In [92]:
dataset = [train_df, test_df]

for data in dataset:
    data['Date of Joining'] = pd.to_datetime(data['Date of Joining'])
    data['Gender'] = [1 if (gender == 'Male') else 0  for gender in data.Gender]
    data['Company Type'] = [1 if (ctype == 'Service') else 0  for ctype in data['Company Type']]
    data['WFH Setup Available'] = [1 if (wfh == 'Yes') else 0  for wfh in data['WFH Setup Available']]
    data['JobDuration'] = [get_days(d, '2009-2-1') for d in data['Date of Joining']]


# train_df['Date of Joining'] = pd.to_datetime(train_df['Date of Joining'])
# train_df['Gender'] = [1 if (gender == 'Male') else 0  for gender in train_df.Gender]
# train_df['Company Type'] = [1 if (ctype == 'Service') else 0  for ctype in train_df['Company Type']]
# train_df['WFH Setup Available'] = [1 if (wfh == 'Yes') else 0  for wfh in train_df['WFH Setup Available']]
# train_df['JobDuration'] = [get_days(d, '2009-2-1') for d in train_df['Date of Joining']]

In [94]:
test_df

,Employee ID,Date of Joining,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,JobDuration
0,fffe31003300390039003000,2008-12-10,0,1,0,2.0,5.0,7.7,53
1,fffe31003300310037003800,2008-08-14,0,0,1,1.0,2.0,5.2,171
2,fffe33003400380035003900,2008-11-13,1,0,1,1.0,3.0,5.9,80
3,fffe3100370039003200,2008-02-07,0,1,0,3.0,6.0,4.6,360
4,fffe32003600390036003700,2008-07-17,0,0,0,2.0,5.0,6.4,199
...,...,...,...,...,...,...,...,...,...
12245,fffe3900310034003700,2008-10-02,0,1,1,1.0,2.0,6.1,122
12246,fffe32003600330034003000,2008-03-31,0,0,1,2.0,4.0,5.9,307
12247,fffe31003800340039003000,2008-02-12,1,1,0,4.0,7.0,9.6,355
12248,fffe32003600380031003800,2008-02-06,1,1,0,3.0,6.0,6.7,361


In [55]:
train_df['Date of Joining'].min(), train_df['Date of Joining'].max()

('2008-01-01', '2008-12-31')

In [56]:
test_df['Date of Joining'].min(), test_df['Date of Joining'].max()

('2008-01-01', '2008-12-31')

In [57]:
train_df

,Employee ID,Date of Joining,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate,JobDuration
0,fffe32003000360033003200,2008-09-30,0,Service,No,2.0,3.0,3.8,0.16,124
1,fffe3700360033003500,2008-11-30,1,Service,Yes,1.0,2.0,5.0,0.36,63
3,fffe32003400380032003900,2008-11-03,1,Service,Yes,1.0,1.0,2.6,0.20,90
4,fffe31003900340031003600,2008-07-24,0,Service,No,3.0,7.0,6.9,0.52,192
5,fffe3300350037003500,2008-11-26,1,Product,Yes,2.0,4.0,3.6,0.29,67
...,...,...,...,...,...,...,...,...,...,...
22743,fffe3300390030003600,2008-12-15,0,Product,Yes,1.0,3.0,6.0,0.48,48
22744,fffe32003500370033003200,2008-05-27,1,Product,No,3.0,7.0,6.2,0.54,250
22746,fffe33003000350031003800,2008-01-19,0,Product,Yes,3.0,6.0,6.7,0.59,379
22748,fffe33003300320036003900,2008-01-10,0,Service,No,2.0,5.0,5.9,0.52,388


In [58]:
train_df['Company Type'].value_counts()

Service    12174
Product     6416
Name: Company Type, dtype: int64

# Data Staging


In [100]:
# Selected features for training
features = ['Gender',   'JobDuration',  'Company Type',	'WFH Setup Available', 	
            'Designation', 'Resource Allocation',	'Mental Fatigue Score']

# Saving Normalizing Parameters for future use with train and test set
feature_min = train_df[features].min().values
feature_max = train_df[features].max().values

# Normalization 
train_df_norm = (train_df[features] - feature_min)/ (feature_max - feature_min)
X_test = (test_df[features] - feature_min)/ (feature_max - feature_min)

y = train_df['Burn Rate']
X_train, X_valid, y_train, y_valid = train_test_split( train_df_norm, y, test_size=0.2, random_state=42)

# Modeling LightGBM


In [96]:
# LGBM Parameters
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['l2', 'auc'],
    'learning_rate': 0.005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 8,
    "num_leaves": 128,  
    "max_bin": 512,
    "num_iterations": 100000,
    "n_estimators": 1000
}


gbm = lgb.LGBMRegressor(**hyper_params)




In [99]:
gbm.fit(X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='l1',
        early_stopping_rounds=1000)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l2: 0.0380535	valid_0's auc: 0.996956	valid_0's l1: 0.155763
Training until validation scores don't improve for 1000 rounds.
[2]	valid_0's l2: 0.0377015	valid_0's auc: 0.996956	valid_0's l1: 0.15504
[3]	valid_0's l2: 0.037353	valid_0's auc: 0.996956	valid_0's l1: 0.154322
[4]	valid_0's l2: 0.037008	valid_0's auc: 0.996956	valid_0's l1: 0.153607
[5]	valid_0's l2: 0.0366664	valid_0's auc: 0.996956	valid_0's l1: 0.152897
[6]	valid_0's l2: 0.036328	valid_0's auc: 0.996956	valid_0's l1: 0.152192
[7]	valid_0's l2: 0.0359966	valid_0's auc: 0.996956	valid_0's l1: 0.151498
[8]	valid_0's l2: 0.0356651	valid_0's auc: 0.996956	valid_0's l1: 0.150802
[9]	valid_0's l2: 0.0353402	valid_0's auc: 0.996956	valid_0's l1: 0.150116
[10]	valid_0's l2: 0.0350152	valid_0's auc: 0.996956	valid_0's l1: 0.149426
[11]	valid_0's l2: 0.0346918	valid_0's auc: 0.997456	valid_0's l1: 0.148739
[12]	valid_0's l2: 0.0343725	valid_0's auc: 0.997456	valid_0's l1: 0.148056
[13]	valid_0's l2: 0.0340563	valid_0'

LGBMRegressor(bagging_fraction=0.7, bagging_freq=10, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
              importance_type='split', learning_rate=0.005, max_bin=512,
              max_depth=8, metric=['l2', 'auc'], min_child_samples=20,
              min_child_weight=0.001, min_split_gain=0.0, n_estimators=1000,
              n_jobs=-1, num_iterations=100000, num_leaves=128,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, task='train',
              verbose=0)

In [101]:
y_test_predicted = gbm.predict(X_test, num_iteration=gbm.best_iteration_)

In [102]:
y_test_predicted

array([0.59353777, 0.35169154, 0.43118669, ..., 0.8032402 , 0.56748319,
       0.1494145 ])

In [84]:
y_pred

array([0.27410108, 0.41063702, 0.36560807, ..., 0.42829171, 0.54964485,
       0.46368896])

In [103]:
submission = pd.DataFrame({'Employee ID' : test_df['Employee ID'], 'Burn Rate': y_test_predicted})
submission.to_csv('submission.csv')